In [10]:
import os
import sys
scriptPath = os.path.abspath('')
parentPath = os.path.join(scriptPath, '..')
sys.path.append(parentPath)

In [20]:
from db import DB
import plotly.express as pex
import plotly.graph_objects as go
import pandas as pd

In [12]:
db = DB()
file = db.fs.getOpt(op='LastDB')
db.openDB(file)

When setting jedzenie, could not get some rows belonging already to other categories: {'jedzenie na miescie'}
When setting zdrowie, could not get some rows belonging already to other categories: {'jedzenie'}
opened db: /home/mi/Dropbox/prog/python/banking/bank.s3db


True

In [13]:
selCat = 'Grandpa'
ignoreCat = 'wlasne'
incomeCat = 'dochody'

outcomeCat = "not " + incomeCat

allCats = db.tree.allChild(selCat)
selLev=allCats[selCat]
# required categories only
cats=[cat for cat,lev in allCats.items() if lev==selLev+1]
if ignoreCat in cats:
    cats.remove(ignoreCat)
if incomeCat in cats:
    cats.remove(incomeCat)

catsKids = {cat: db.tree.child(cat) for cat in cats}
kidsCats = {}
for kid,l in allCats.items():
    par = [par for par in catsKids if kid in catsKids[par]]
    if par:
        kidsCats[kid] = par[0]

for par in catsKids.keys():
    kidsCats[par]=par


startDate, endDate = db.dataRange()
# ['2021-09-01', '2022-01-17']

In [14]:
# categories
df = db.op.get(list(kidsCats.keys()))
start = df.loc[:, db.DATA_OP] >= pd.to_datetime(startDate)
end = df.loc[:, db.DATA_OP] <= pd.to_datetime(endDate)
df = df.loc[start & end, [db.DATA_OP, db.AMOUNT, db.CATEGORY]]

df.loc[:,'parCat'] = [kidsCats[i] for i in df.loc[:,db.CATEGORY]]


In [15]:
# bilans
catBilans = list(allCats.keys())
catBilans.remove(ignoreCat)
df_bilans = db.op.get(catBilans)
start = df_bilans.loc[:, db.DATA_OP] >= pd.to_datetime(startDate)
end = df_bilans.loc[:, db.DATA_OP] <= pd.to_datetime(endDate)
df_bilans = df_bilans.loc[start & end, [db.DATA_OP, db.AMOUNT, db.CATEGORY]]

df_bilans.loc[:,'parCat'] = df_bilans.loc[:,db.CATEGORY]

In [17]:
# group
timeGrp = pd.Grouper(key=db.DATA_OP, freq='M')
df = df.groupby([timeGrp, 'parCat']).sum()
df_bilans = df_bilans.groupby(timeGrp).sum()
df.reset_index(inplace=True)
df_bilans.reset_index(inplace=True)

In [31]:
fig = go.Figure()
fig.add_traces(list(pex.bar(df,
                           x=db.DATA_OP,
                           y=db.AMOUNT,
                           color='parCat',
                           barmode='group').select_traces()))

fig.add_traces(list(pex.line(df_bilans,
                               x=db.DATA_OP,
                               y=db.AMOUNT).select_traces()))

fig.show()
